In [2]:
import geopandas as gpd
import sqlite3

import numpy as np
import pandas as pd
from shapely import wkt
import pyogrio
from tqdm import tqdm

data_raw = "C:/Users/eleonore.kong/jll.spear/Nicolas Le Corvec - RGA/lowpass_butterworth_data.db"
swi_data = 'C:/Users/eleonore.kong/Documents/InSAR/SWI/SWI_Package_1969-2023.gpkg'
geopackage_path = "C:/Users/eleonore.kong/Documents/InSAR/DATA/lpf_egms_data.gpkg"
# grids = ['grid_1km', 'grid_500m']

In [3]:
# Connect to the SQLite database
conn = sqlite3.connect(data_raw)

query = "SELECT name FROM sqlite_master WHERE type='table';"

# Execute the query and get the table names
tables = pd.read_sql_query(query, conn)
gdf_swi = pyogrio.read_dataframe(swi_data, layer='data_swi_peak_trough')
gdf_swi = gdf_swi.drop(['SWI_UNIF_MENS3_trough', 'SWI_UNIF_MENS3_peak'], axis=1)
gdf_swi.columns = gdf_swi.columns.str.lower()

rga_df = pyogrio.read_dataframe("C:/Users/eleonore.kong/Documents/InSAR/ARGILES/Risk_ARG.gpkg")
rga_df['NIVEAU'] = rga_df['NIVEAU'].fillna(0)

tables_list = tables['name'].tolist()
print(tables_list)

['lowpass_Butterworth_output_EGMS_L3_E35N30_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E34N30_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E33N27_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E33N23_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E32N28_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E32N29_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E34N29_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E34N25_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E33N29_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E36N30_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E36N26_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E34N26_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E34N24_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E35N23_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E36N25_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E36N28_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E35N29_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E36N22_100km_U', 'lowpass_Butterworth_output_EGMS_L3_E36N27_10

In [4]:
tables_list[65]

'lowpass_Butterworth_output_EGMS_L3_E35N21_100km_U'

In [5]:
# Query the table
print('Found {} tables'.format(len(tables_list)))
i=1

for table_name in tqdm(tables_list[65:]):

    query = "SELECT * FROM " + table_name 

    # Read the data into a Pandas DataFrame
    df = pd.read_sql_query(query, conn)
    df.sort_values(by=['pid', 'Date'])
    df['Date'] = pd.to_datetime(df['Date'])
    df['year'] = df['Date'].dt.year
    df['month'] = df['Date'].dt.month
    df['time_gap'] = df['Date'].diff().apply(lambda x: x / pd.Timedelta(days=30))
    season_maper = {1:'winter', 2:'winter', 3:'winter', 4:'winter', 5:'inter',
                    6:'summer', 7:'summer', 8:'summer', 9:'summer', 
                    10:'summer', 11:'inter', 12:'winter'}
    df['season'] = df['month'].map(season_maper)

    df.loc[df['time_gap'] < 0, 'time_gap'] = np.nan
    stats = df[['pid', 'time_gap']].groupby('pid').median().reset_index()
    stats = stats[(stats['time_gap'] >= 4) & (stats['time_gap'] <= 7)]
    pid_list = stats['pid'].unique()
    df = df[df['pid'].isin(pid_list)]
    df['value_diff'] = df['Value'].diff()
    df.loc[df['time_gap'].isna(), 'value_diff'] = np.nan
    
    stats = df[['pid', 'Type', 'season', 'Value']].groupby(['pid', 'Type', 'season']).count()
    stats = stats.unstack(['Type', 'season'])
    stats.columns = [f'{season}_{type}' for value, season, type in stats.columns]
    stats.reset_index(inplace=True)
    stats['is_rga'] = (stats['Peak_winter'] > 4) & (stats['Trough_summer'] > 4)
    is_rga_maper = dict(zip(stats['pid'], stats['is_rga']))
    df['is_rga'] = df['pid'].map(is_rga_maper)
    # df.drop('Date', axis=1, inplace=True)

    # df = df[df['is_rga'] == True]
    df.reset_index(inplace=True)
    df.loc[df['month'] == 12, 'year'] = df['year'] + 1
    df['value_type'] = df['season'] +'_'+ df['year'].astype(str)
    # keep = (df['season'] == 'winter') & (df['Type'] == 'Peak') | (df['season'] == 'summer') & (df['Type'] == 'Trough')
    df['value_diff_abs'] = abs(df['value_diff'])
    # df = df[keep]

    # Convert to a GeoDataFrame
    df['geometry'] = df['geometry'].apply(wkt.loads)
    gdf = gpd.GeoDataFrame(df, geometry='geometry', crs='EPSG:4326').to_crs(epsg='2154')
    gdf = gdf.reset_index(drop=True)
    
    link_insar_swi = gdf[['geometry']].sjoin(gdf_swi)
    if len(link_insar_swi) > 0:
        link_insar_swi.drop('index_right', axis=1, inplace=True)
        gdf = gdf.merge(link_insar_swi, on=['geometry', 'year'])
        gdf = gdf.drop_duplicates()
        gdf['gap_insar_swi_trough'] = gdf['month'] - gdf['month_trough']
        gdf['gap_insar_swi_peak'] = gdf['month'] - gdf['month_peak']
        gdf['is_swi_close'] = (gdf['gap_insar_swi_trough'] >= -2) & (gdf['gap_insar_swi_trough'] <= 2) & (gdf['Type'] == 'Trough') | (gdf['gap_insar_swi_peak'] >= -2) & (gdf['gap_insar_swi_peak'] <= 2) & (gdf['Type'] == 'Peak')
        stats = gdf[['pid', 'is_swi_close', 'gap_insar_swi_trough']].groupby(['pid', 'is_swi_close']).count()
        stats = stats.unstack(['is_swi_close'])
        stats.columns = [f'{type}' for _, type in stats.columns]
        stats.reset_index(inplace=True)
        stats['is_swi_close'] = (stats['True'] > stats['False'])
        is_swi_close_maper = dict(zip(stats['pid'], stats['is_swi_close']))
        gdf['is_swi_close'] = gdf['pid'].map(is_swi_close_maper)
    
        gdf = gdf.sjoin(rga_df[['geometry', 'NIVEAU']], how='left')
    
        if i == 0:
            pyogrio.write_dataframe(gdf, geopackage_path, layer="data_all", driver="GPKG")
        else:
            pyogrio.write_dataframe(gdf, geopackage_path, layer="data_all", driver="GPKG", append=True)
    
    else:
        continue
    i+=1

    
# Close the connection
conn.close()

Found 81 tables


100%|██████████| 16/16 [4:21:29<00:00, 980.57s/it]   
